**420-A52-SF - Algorithmes d'apprentissage supervisé - Hiver 2021 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2021 Mikaël Swawola
<br/>
![Travaux Pratiques - Arbres de décision](static/15-tp-banner.png)
<br/>
**Objectif:** cette séance de travaux pratiques a pour objectif la mise en oeuvre des arbres de régression et de classification. Les jeux de données utilisés seront **Hitters** (Baseball) et **Heart**

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd

## Exercice 1 - Arbres de régression

### 1-1 Chargement du jeu de données Hitters et exploration sommaire

In [11]:
dfHit = pd.read_csv("../../data/Hitters.csv")
dfHeart = pd.read_csv("../../data/Heart.csv")
dfHit.head()

,Unnamed: 0,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,...,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,-Andy Allanson,293,66,1,30,29,14,1,293,66,...,30,29,14,A,E,446,33,20,NaN,A
1,-Alan Ashby,315,81,7,24,38,39,14,3449,835,...,321,414,375,N,W,632,43,10,475.0,N
2,-Alvin Davis,479,130,18,66,72,76,3,1624,457,...,224,266,263,A,W,880,82,14,480.0,A
3,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,...,828,838,354,N,E,200,11,3,500.0,N
4,-Andres Galarraga,321,87,10,39,42,30,2,396,101,...,48,46,33,N,E,805,40,4,91.5,N


In [12]:
#dfHit.isnull().sum()
dfHeart.isnull().sum()

Unnamed: 0    0
Age           0
Sex           0
ChestPain     0
RestBP        0
Chol          0
Fbs           0
RestECG       0
MaxHR         0
ExAng         0
Oldpeak       0
Slope         0
Ca            4
Thal          2
AHD           0
dtype: int64

In [20]:
dfHeart = dfHeart.loc[:, ~dfHeart.columns.str.contains('^Unnamed')]
dfHeart.head()

#dfHit = dfHit.loc[:, ~dfHit.columns.str.contains('^Unnamed')]
#dfHit.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [28]:
def getDfNum(pDf):
    return  pd.get_dummies(pDf, drop_first=True, prefix_sep="_")

In [37]:
def getDfXDfY(pDf, pColY):
    dfY = pd.DataFrame(pDf[pColY])
    dfX = pDf.drop([pColY], axis=1)
    return dfX, dfY


,Salary
1,475.0
2,480.0
3,500.0
4,91.5
5,750.0


### 1-2 Suppression des observations ayant des valeurs manquantes

In [32]:
dfHit.dropna(inplace=True)
dfHeart.dropna(inplace=True)

In [7]:
#df.isnull().sum()

Unnamed: 0    0
AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary        0
NewLeague     0
dtype: int64

### 1-3 Préparation de la structure de données

In [42]:
dfHeartNum = getDfNum(dfHeart)
dfHitNum = getDfNum(dfHit)


dfXHit, dfYHit = getDfXDfY(dfHitNum, 'Salary')
dfXHeart, dfYHeart = getDfXDfY(dfHeartNum, 'AHD_Yes')



In [82]:
from sklearn.model_selection import train_test_split




### 1-4 Validation croisée Holdout 

In [98]:
### definir baseline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn import tree
import matplotlib.pyplot as plt

def analyze_regression(pDfX, pDfY):
    X_train, X_test, y_train, y_test = train_test_split(pDfX, pDfY
    #, stratify=True
    , test_size=0.3
    , shuffle=True
    , random_state=0
    )
    dt = tree.DecisionTreeRegressor(criterion='squared_error')
    dt.fit(X_train, y_train)
    dt_scores = cross_val_score(dt, X_train, y_train, cv=10)

    
    
    print (mean_squared_error(y_train, dt.predict(X_train),  squared=False))
    print(dt.score(X_train, y_train))
    print("dt_scores: ", dt_scores)

    text_representation = tree.export_text(dt)
    print(text_representation)


    #fig = plt.figure(figsize=(25,20))
    #_ = tree.plot_tree(dt, 
    #               feature_names=pDfX.columns,  
    #               class_names=pDfY.columns,  
    #               filled=True)

1.0
dt_scores:  [ 0.58193237 -0.60939669  0.36795957  0.18665888  0.57487601  0.35065852
  0.60701246 -1.66436898  0.45944088 -0.18732236]


### 1-5 Mise en oeuvre de l'arbre de régression

In [99]:
analyze_regression(dfXHit, dfYHit)


0.0
1.0
dt_scores:  [ 0.52557192 -0.55402996  0.35367221  0.37248829  0.61006364  0.37385357
  0.6892302  -3.18391914 -0.14408361 -0.30010682]
|--- feature_8 <= 462.00
|   |--- feature_3 <= 4.00
|   |   |--- value: [2127.33]
|   |--- feature_3 >  4.00
|   |   |--- feature_7 <= 1257.50
|   |   |   |--- feature_8 <= 182.50
|   |   |   |   |--- feature_11 <= 51.00
|   |   |   |   |   |--- feature_10 <= 17.50
|   |   |   |   |   |   |--- value: [190.00]
|   |   |   |   |   |--- feature_10 >  17.50
|   |   |   |   |   |   |--- feature_8 <= 82.00
|   |   |   |   |   |   |   |--- feature_9 <= 8.00
|   |   |   |   |   |   |   |   |--- feature_13 <= 384.00
|   |   |   |   |   |   |   |   |   |--- feature_18 <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- feature_0 <= 212.00
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |   |   |   |   |   |   |--- feature_0 >  212.00
|   |   |   |   |   |   |   |   |   |   |   |--- value: [70.00]
|   |   |   |

[class sklearn.tree.DecisionTreeRegressor(criterion='mse', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, presort='deprecated', ccp_alpha=0.0)](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

### 1-6 Évaluation du modèle

### 1-7 Visualisation de l'arbre

In [ ]:
text_representation = tree.export_text(clf)
print(text_representation)

In [75]:
#import sys
#!{sys.executable} -m pip install graphviz
#import graphviz 
#dot_data = tree.export_graphviz(clf, out_file='tree.dot') 

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 150.4 kB/s eta 0:00:0031m7.6 MB/s eta 0:00:01


### 1-8 Application de l'élagage et ré-évaluation du modèle

### 1-9 Visualisation de l'arbre élagué

## Exercice 2 - Arbres de classification

### 2-1 Chargement et préparation du jeu de données Heart

### 2-2 Préparation de la structure de données

### 2-3 Validation croisée Holdout 

### 2-4 Mise en oeuvre de l'arbre de classification

[class sklearn.tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort='deprecated', ccp_alpha=0.0)](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

### 2-5 Évaluation du modèle

### 2-6 Application de l'élagage et ré-évaluation du modèle

### 2-7 Visualisation de l'arbre